In [ ]:
import os
import re
import sys
import matplotlib

import numpy as np
import scipy as sp
import pandas as pd
import firescipy as fsp

import matplotlib.pyplot as plt

In [ ]:
###########################################################################
## ! Use the 'requirements.txt' to create a virtual Python environment ! ##
###########################################################################

# Package Versions
# ----------------
# Python version: 3.13.3 (tags/v3.13.3:6280bb5, Apr  8 2025, 14:47:33) [MSC v.1943 64 bit (AMD64)]
# Python path: F:\PhD\BESKID\Pyrolysis_BESKID_Paper\BESKID_Paper_venv\venv_BESKID_Paper\Scripts\python.exe
# Numpy version: 2.3.2
# SciPy version: 1.16.1
# Pandas version: 2.3.1
# FireSciPy version: 0.0.5
# Matplotlib version: 3.10.5
# RegEx version: 2.2.1


print('Package Versions')
print('----------------')
print('Python version: {}'.format(sys.version))
print('Python path: {}'.format(sys.executable))
print('Numpy version: {}'.format(np.__version__))
print('SciPy version: {}'.format(sp.__version__))
print('Pandas version: {}'.format(pd.__version__))
print('FireSciPy version: {}'.format(fsp.__version__))
print('Matplotlib version: {}'.format(matplotlib.__version__))
print('RegEx version: {}'.format(re.__version__))


In [ ]:
# General information.

exp_tubs_root = os.path.join("..", "Experiments", "TUBS")




# Order of plot colors
plt_colors = ["tab:blue", "tab:orange", "tab:green", 
              "tab:red", "tab:purple", "tab:brown", 
              "tab:pink", "tab:gray", "tab:olive", 
              "tab:cyan"]

# Basic plot settings
plt.rcParams.update({
    'axes.axisbelow': True,     # Keep grid behind plots
    'figure.autolayout': True,  # Equivalent to calling tight_layout()
    'axes.facecolor': 'white',  # Prevents transparent background
    'grid.alpha': 0.6,          # Makes gridlines more readable
})


# Directory used to collect the output produced by this notebook.
output_dir = "AssessExperiments_Output"
if not os.path.isdir(output_dir):
    os.mkdir(output_dir)
    print("* Output directory created.")
else:
    print("* Output directory already exists.")


# Micro-Scale Experiments

Focus on the STA and MCC experiments conducted by iBMB.
- [DOI: 10.24355/dbbs.084-202508070641-0](https://doi.org/10.24355/dbbs.084-202508070641-0)


In [ ]:
# Dictionary to store data
tubs_microscale_data = dict()

# Define regular expression (regex) patterns for different experiment types
patterns = {
#     "microscale": re.compile(r'(?P<setup>tga|mcc)_(?P<temp_program>isotherm|dynamic)_(?P<atmosphere>n2)_(iso|dyn?P<temp>\d+(?:[-.]\d+)?)_(?P<sample>piece|powder)_(?P<mass>\d+(?:[-.]\d+)?mg)_(?P<rep>\d+(?:[-.]\d+)?)?\.(?:csv|txt)'),
    "microscale": re.compile(
        r'(?P<setup>tga|mcc)_'
        r'(?P<temp_program>isotherm|dynamic)_'
#         r'(?P<atmosphere>n2)_'
#         r'(?P<atmosphere>n2|n2high|n2low)(?P<purge_flow>\d+(?:[-.]\d+)?)_'
#         r'(?P<atmosphere>n2(?:high|low))(?P<purge_flow>\d+(?:[.-]\d+)?)?(?:_|$)'
        r'(?P<atmosphere>n2(?:high|low)?)(?P<purge_flow>\d+(?:[.-]\d+)?)?(?:_|$)'
        r'(?P<temp_prefix>iso|dyn)(?P<temp_number>\d+(?:[-.]\d+)?)_'
        r'(?P<sample>piece|powder|powdertubs)_'
        r'(?P<mass>\d+(?:[-.]\d+)?mg)_'
        r'(?P<rep>r\d+)\.(?:csv|txt)',
        re.IGNORECASE
    )
}


# Keep track of processed files to make sure all are captured
skipped_files = list()

# Loop through experimental setups
for exp_setup in os.listdir(exp_tubs_root):
    exp_setup_path = os.path.join(exp_tubs_root, exp_setup) 
    if not os.path.isdir(exp_setup_path):
        continue  # Skip non-directories

    print(f"* Processing {exp_setup} ...")

    # Loop through files in experimental setup directory
    for file_label in os.listdir(exp_setup_path):
        print(f"  {file_label}")
        
        file_info = None
        matched_type = None

        # Try matching against different regex patterns
        for exp_type, pattern in patterns.items():
            match = pattern.match(file_label)
            if match:
                file_info = match.groupdict()
                matched_type = exp_type
                break  # Stop at the first match
        
        # If no valid format was found, skip the file
        if not file_info:
            skipped_files.append(file_label)
            continue
        
        # If no proper repetition label is provided, show message
        if not file_info.get("rep"):
            print(f"** File label '{file_label}' is missing a repetiton number --> file skipped!")
            skipped_files.append(file_label)
            continue
        
        # Extract metadata
        exp_setup = file_info.get("setup", "tga")  # Default to "tga" if missing
        rep_label = f"Rep_{file_info['rep'][1:]}"

        if matched_type == "microscale":
            atmosphere = f"{file_info['atmosphere']}"
            purge_flow = f"{file_info['purge_flow']}"
            
            if purge_flow == "None":
                atmo_label = f"{file_info['atmosphere']}"
            else:
                atmo_label = f"{file_info['atmosphere']}{file_info['purge_flow']}"
            print("***", atmo_label)
#             purge_flow = f"{file_info['purge_flow']}"
#             print(purge_flow)
            temp_program = file_info["temp_program"]
            sample_type = file_info["sample"]
            sample_mass = file_info["mass"]
            unit_map = {"iso": "C", "dyn": "Kmin"}
            temp_label = f'{file_info["temp_number"]}{unit_map[file_info["temp_prefix"]]}'            
            keys = [exp_setup, atmo_label, sample_type, sample_mass, temp_program, temp_label, rep_label]

        # Read the CSV
        exp_path = os.path.join(exp_setup_path, file_label)
        exp_df = pd.read_csv(exp_path, header=0, skiprows=[1], 
                             delimiter="\t", encoding='cp1252')
        
#         print(exp_df.head(2))

        # Store in nested dictionary
        fsp.utils.store_in_nested_dict(tubs_microscale_data, exp_df, keys)

        print(f"  Loaded {file_label} -> {keys}")
        
    print()

# Display skipped files
if skipped_files:
    print("* The following files were skipped:\n" + "\n".join(skipped_files))

print("\n* Done.")


# Store Data from Experiments and Kinetics Computations


In [ ]:
fzj_powder_masses = ["1mg", "2mg", "3mg", "6mg", "12mg"]
fzj_powder = tubs_microscale_data["tga"]['n2']['powder']

fzj_powder["1mg"]["dynamic"]["10Kmin"]["Rep_1"].head()


In [ ]:
# Desired sample properties
sample_masses=["1mg", "2mg", "3mg"]
fzj_powder = tubs_microscale_data["tga"]['n2']['powder']

# Provide nominal temperature program
hrs = {
    "5Kmin": [5.0, "K/min"],
    "10Kmin": [10, "K/min"],
    "20Kmin": [20, "K/min"],
    "30Kmin": [30, "K/min"],
    "60Kmin": [60, "K/min"]
}


# Initialise kinetics data collection
BESKID_data = {
    "dynamic": dict(),
    "isotherm": dict()}


for sample_mass in sample_masses:
    # Initialise investigation.
    investigation = fsp.pyrolysis.kinetics.initialize_investigation_skeleton(
        material=f"PMMA DE 6mm ({sample_mass}), BESKID",  # Sample material
        investigator="iBMB (TUBS)",  # Conducted experiments
        instrument="TGA/DSC 3+, Mettler Toledo", 
        date="Stardate: 42.69", 
        notes="Constant heating rates, sample mass 1mg",
        signal={"name": "Mass", "unit": "mg"}
    )
    
    # Get constant heating rate data (dynamic)
    dyn_exp = fzj_powder[sample_mass]["dynamic"]
    for hr_label in hrs:
        
        # Go over the repetitions
        for rep_label in dyn_exp[hr_label]:
            # Get the data set
            rep_data = dyn_exp[hr_label][rep_label]
            # Adjust temperatures to Kelvin
            rep_data['ts'] = rep_data['ts'] + 273.15
            rep_data['tr'] = rep_data['tr'] + 273.15
            
            # Add raw experiment data set to collection.
            fsp.pyrolysis.kinetics.add_constant_heating_rate_tga(
                database=investigation, 
                condition=hr_label, 
                repetition=rep_label, 
                raw_data=rep_data, 
                data_type="integral", 
                set_value=hrs[hr_label])
    
    
        # Column header mapping - adjust to your file structure!
        # This is the standard, i.e. column_mapping=None
        column_labels = {
            'time': 't', 
            'temp': 'ts', 
            'signal': 'weight'}
        
        # Combine repetitions
        fsp.pyrolysis.kinetics.combine_repetitions(
            database=investigation, 
            condition=hr_label, 
            column_mapping=column_labels)
        
    # Compute the conversions of the combined data
    fsp.pyrolysis.kinetics.compute_conversion(
        database=investigation,
        condition="all",  # Can also compute for individual temperature programs, e.g. "5Kmin"
        setup="constant_heating_rate")

    # Compute the conversion fractions, necessary for KAS method
    # Default conversion levels.
    # from 0.05 to 0.95, in 0.025 steps
    conversion_levels = np.linspace(0.01, 0.99, 99)  # Δα = 1.0

    fsp.pyrolysis.kinetics.compute_conversion_levels(
        database=investigation, 
        desired_levels=conversion_levels,
        setup="constant_heating_rate", 
        condition="all")


    # Compute estimates for activation energy and pre-exponenital factor
    # using Kissinger–Akahira–Sunose with Starink improvement
    fsp.pyrolysis.kinetics.compute_Ea_KAS(database=investigation)

    # Collect results.
    BESKID_data["dynamic"][sample_mass] = investigation


# Check results
Ea_results_KAS_BESKID = BESKID_data["dynamic"]["1mg"]["experiments"]["TGA"]["Ea_results_KAS"]
Ea_results_KAS_BESKID.head()


In [ ]:
# Write combined repetitions to disk (average, std)
sample_masses = ["1mg", "2mg", "3mg"]

for sample_mass in sample_masses:
    # sample_mass = '1mg'
    constant_heating_rate = BESKID_data["dynamic"][sample_mass]["experiments"]["TGA"]["constant_heating_rate"]
    
    for hr_label in list(constant_heating_rate):
        # hr_label = "10Kmin"
        combined_df = constant_heating_rate[hr_label]["combined"]
        
        file_name = os.path.join(output_dir, f"TGA_{sample_mass}_{hr_label}.csv")
        combined_df.to_csv(file_name, sep=',', encoding='utf-8', index=False, header=True)


In [ ]:
# Visualise Ea over alpha
institute = "TUBS"
sample_mass = "1mg"


fig, ax1 = plt.subplots()

for mass_id, sample_mass in enumerate(sample_masses):
    # Get the Ea and convert to kJ/mol.
    Ea_results_KAS = BESKID_data["dynamic"][sample_mass]["experiments"]["TGA"]["Ea_results_KAS"]
    y_data = Ea_results_KAS["Ea"]/1000

    # Plot the Ea against conversion.
    start = 1  # Skip the 0.05 because of being outlier
    stop = None
    ax1.scatter(Ea_results_KAS["Conversion"][start:stop],
                y_data[start:stop],
#                 marker='o', s=17,
                marker=".", s=42,
                facecolors='none',
                edgecolors=plt_colors[mass_id],
                label=f"Exp.: {institute}, {sample_mass}, KAS")


# Shaded areas to indicate first/last 5 % (typically excluded)
x_min = -0.025
x_max = 1.025
ax1.axvspan(x_min, 0.05, color='gray', alpha=0.3, label="5%")
ax1.axvspan(0.95, x_max, color='gray', alpha=0.3)


# Plot meta data.
# plt.title(f"Activation Energy (KAS)")
plt.xlabel("Conversion ($\\alpha$) / -")
plt.ylabel("Activation Energy (E$_a$) / kJ/mol")

plt.xlim(left=-0.025, right=1.025)
plt.ylim(bottom=160, top=240)

plt.rcParams.update({'font.size': 12})

plt.tight_layout()
plt.legend(loc = "lower right")
plt.grid()


# Save image.
plot_label = f"{institute}_Ea_Estimate_KAS.png"
plot_path = os.path.join(output_dir, plot_label)
plt.savefig(plot_path, dpi=320, bbox_inches='tight', facecolor='w')
